# Task 2: Use SageMaker Clarify
	
In this lab, you use Amazon SageMaker Clarify to detect bias in pre-training data and post-training models and access explainability reports.


## Task 2.1: Environment setup

Install packages and dependencies.

In [ ]:
#install-dependencies

import boto3
import io
import json
import math
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import sagemaker
import sagemaker_datawrangler
import sys
import time
import zipfile

from IPython.display import display
from IPython.display import Image
from sagemaker import clarify
from sagemaker import Session
from sagemaker.inputs import TrainingInput
from sagemaker.s3 import S3Uploader
from sklearn.model_selection import train_test_split
from time import gmtime, strftime

s3_client = boto3.client("s3")
session = Session()
bucket = session.default_bucket()
prefix = 'sagemaker/lab_8'
role = sagemaker.get_execution_role()

Next, import, split, and upload the dataset.

In [ ]:
#prepare-dataset

lab_test_data = pd.read_csv('adult_data_processed.csv')

# Split the dataset
train_data, validation_data, test_data = np.split(
    lab_test_data.sample(frac=1, random_state=1729),
    [int(0.7 * len(lab_test_data)), int(0.9 * len(lab_test_data))],
)

train_data.to_csv('train_data.csv', index=False, header=False)
validation_data.to_csv('validation_data.csv', index=False, header=False)
test_data.to_csv('test_data.csv', index=False, header=False)

# Upload the Dataset to S3
from sagemaker.s3 import S3Uploader
from sagemaker.inputs import TrainingInput

sagemaker_session = sagemaker.Session()

train_path = S3Uploader.upload('train_data.csv', 's3://{}/{}'.format(bucket, prefix))
validation_path = S3Uploader.upload('validation_data.csv', 's3://{}/{}'.format(bucket, prefix))
test_path = S3Uploader.upload('test_data.csv', 's3://{}/{}'.format(bucket, prefix))

train_input = TrainingInput(train_path, content_type='text/csv')
validation_input = TrainingInput(validation_path, content_type='text/csv')
test_input = TrainingInput(validation_path, content_type='text/csv')

data_inputs = {
    'train': train_input,
    'validation': validation_input
}

Now, train the XGBoost model. You use this trained model for the SageMaker Clarify ModelConfig. The training takes approximately 3-4 minutes to complete.

In [ ]:
#train-model

# Retrieve the container image
container = sagemaker.image_uris.retrieve(
    region=boto3.Session().region_name, 
    framework='xgboost', 
    version='1.5-1'
)
# Set up the estimator
xgb = sagemaker.estimator.Estimator(
    container,
    role, 
    instance_count=1, 
    instance_type='ml.m5.xlarge',
    output_path='s3://{}/{}/output'.format(bucket, prefix),
    sagemaker_session=sagemaker_session
)
# Set the hyperparameters
xgb.set_hyperparameters(
    max_depth=5, 
    eta=0.2, 
    gamma=4, 
    min_child_weight=6,
    subsample=0.8, 
    verbosity=1, 
    objective='binary:logistic', 
    num_round=800
)

# Train the model
xgb.fit(
    inputs = data_inputs
) 

## Task 2.2: Set the SageMaker Clarify job and access the bias report

To use [SageMaker Clarify](https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-configure-processing-jobs.html), you first create a model from your training job. Then you set up the necessary configurations and run the SageMaker Clarify job on your trained model. In this task, you complete the following:

- Create the model.
- Enable SageMaker Clarify.
- Run the bias report.
- Access the report.

### Task 2.2.1: Create the model

Create a model from the training job to use with SageMaker Clarify.

In [ ]:
#create-clarify-model

model_name = "lab-8-clarify-model"
model = xgb.create_model(name=model_name)
container_def = model.prepare_container_def()
session.create_model(model_name, role, container_def)

### Task 2.2.2 Enable SageMaker Clarify

Now, to begin configuration, enable SageMaker Clarify.

In [ ]:
#enable-clarify

clarify_processor = clarify.SageMakerClarifyProcessor(
    role=role, 
    instance_count=1, 
    instance_type="ml.m5.xlarge", 
    sagemaker_session=session
)

With SageMaker Clarify, you can detect potential pre-training bias in your data and post-training bias in your models through various metrics. Refer to [metrics](https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-processing-job-configure-analysis.html) for more information about configuring the clarify analysis.

Configure the **DataConfig** object to communicate data I/O information to SageMaker Clarify. Here, you specify the location of the input dataset, the location for the output, the income (**label**) column, the header names, and the dataset type.

In [ ]:
#define-data-config

bias_report_output_path = "s3://{}/{}/clarify-bias".format(bucket, prefix)
bias_data_config = clarify.DataConfig(
    s3_data_input_path=train_path,
    s3_output_path=bias_report_output_path,
    label="income",
    headers=train_data.columns.to_list(),
    dataset_type="text/csv",
)

Configure the **ModelConfig** object to communicate information about your trained model. Set the model name and a temporary dedicated endpoint to avoid additional traffic to your production model (**instance_type** and **instance_count**). Also set an **accept_type** to denote the endpoint response payload format, and a **content_type** to denote the payload format of request to the endpoint.

In [ ]:
#define-model-config

model_config = clarify.ModelConfig(
    model_name=model_name,
    instance_type="ml.m5.xlarge",
    instance_count=1,
    accept_type="text/csv",
    content_type="text/csv",
)

Configure the **probability_threshold** of the **ModelPredictedLabelConfig** to convert the probability of samples to binary labels for bias analysis. Prediction above the threshold is interpreted as label value **1** and below or equal as label value **0**.

In [ ]:
#define-model-predicted-label-config

predictions_config = clarify.ModelPredictedLabelConfig(probability_threshold=0.8)

Configure the **BiasConfig** to denote the sensitive columns (**facets**), potential sensitive features (**facet_values_or_threshold**), and favorable outcomes (**label_values_or_threshold**).

You can denote both categorical and continuous data for **facet_values_or_threshold** and **label_values_or_threshold**. The **sex** and **age** features are categorical.

The goal of this model is to determine if an individual makes 50,000 USD or more, with the positive outcome being favorable. Use **BiasConfig** to provide information on which columns contain the facets with the sensitive group of **Sex**, what the sensitive features might be using **facet_values_or_threshold**, and what the desirable outcomes are using **label_values_or_threshold**. Group by **age** to see if there are any differences depending on the individual's age.

In [ ]:
#define-bias-config

bias_config = clarify.BiasConfig(
    label_values_or_threshold=[1], facet_name="sex", facet_values_or_threshold=[0], group_name="age"
)

### Task 2.2.3: Run the bias report

Create the bias report using your configurations for pre-training and post-training analysis. This step takes approximately 15–20 minutes. While the bias report is being generated, you can discover how SageMaker Clarify computes [Fairness Measures for Machine Learning in Finance](./Fairness.Measures.for.Machine.Learning.in.Finance.pdf).

In [ ]:
#run-bias-report

clarify_processor.run_bias(
    data_config=bias_data_config,
    bias_config=bias_config,
    model_config=model_config,
    model_predicted_label_config=predictions_config,
    pre_training_methods="all",
    post_training_methods="all",
)

### Task 2.2.4: Access the bias report

In SageMaker Studio, you can view the results under the experiments tab.

The next step opens a new tab in SageMaker Studio. To follow these directions, use one of the following options:
- **Option 1:** View the tabs side by side. To create a split screen view from the main SageMaker Studio window, either drag the **lab_8.ipynb** tab to the side or choose (right-click) the **lab_8.ipynb** tab and choose **New View for Notebook**. You can now have the directions visible as you explore the bias report.
- **Option 2:** Switch between the SageMaker Studio tabs to follow these instructions. When you are finished exploring the bias report, return to the notebook by choosing the **lab_8.ipynb** tab.

1. In SageMaker Studio, choose the **SageMaker Home** icon.

2. Choose **Experiments**.

SageMaker studio opens the **Experiments** tab.

3. In the **Experiments** tab, select **Unassigned runs** from the left.

4. In the **Unassigned runs** list, select the training job name that contains **clarify-bias-** in the title. 

5. In the **Experiments** tab, select **Bias Reports** from the left.

A bias report of the metrics is available when SageMaker Clarify finishes running the bias job.

6. Wait for SageMaker Clarify to finish running the bias job.

Each bias metric has detailed explanations with examples that you can explore by selecting each value. 

7. Examine the **Class Imbalance** and **Disparate (Adverse) Impact (DI)** detailed explanations by choosing the arrows next to the bias metrics, and expanding the field. 

When you are finished exploring the bias metrics, continue with the next task.

## Task 2.3: Access explainability reports

In addition to using a bias report, you can use SageMaker Clarify to analyze the local explanations of individual predictions. Create a report to review explainability results for the predictions produced by SageMaker Clarify and analyze key metrics from the report. In this task, you complete the following:

1. Define a Shapley Values (SHAP) configuration.
2. Run the explainability report.
3. Access the explainability report.

### Task 2.3.1: Define a SHAP configuration

You can run an explainability report to get an explanation of why a model made a specific prediction. SageMaker Clarify uses SHAP to generate a report on the model's reasoning for its decisions. Refer to [SHAP Baselines for Explainability](https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-feature-attribute-shap-baselines.html) for more information about SHAP baselines.

The SHAP metrics that are configured are as follows:
- **baseline**: A list of rows, or an Amazon Simple Storage Service (Amazon S3) object URI, to be used as the baseline dataset in the Kernel SHAP algorithm.
- **num_samples**: Number of samples to be used in the Kernel SHAP algorithm. This number determines the size of the generated synthetic dataset to compute the SHAP values.
- **agg_method: mean_abs**: Mean of absolute SHAP values for all instances.
- **save_local_shap_values**: Boolean value to indicate if local SHAP values are to be saved in the output location.

Refer to [SHAP metrics](https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-processing-job-configure-analysis.html) for more information about the metrics used.

In [ ]:
#configure-shap

testing_data, clarify_data = train_test_split(test_data, test_size =0.005)
clarify_data = clarify_data.drop(columns=["income"])
clarify_data.to_csv('clarify_data.csv', index=False, header=False)
clarify_path = S3Uploader.upload('clarify_data.csv', 's3://{}/{}'.format(bucket, prefix))

shap_config = clarify.SHAPConfig(
    baseline=clarify_path,
    num_samples=15,
    agg_method="mean_abs",
    save_local_shap_values=True,
)

explainability_output_path = "s3://{}/{}/clarify-explainability".format(bucket, prefix)
explainability_data_config = clarify.DataConfig(
    s3_data_input_path=clarify_path,
    s3_output_path=explainability_output_path,
    headers=clarify_data.columns.to_list(),
    dataset_type="text/csv",
)

### Task 2.3.2: Run the explainability report

Create the explainability report using your configurations. This step takes approximately 10–15 minutes. While the explainability report is being generated, you can follow along with the job status in SageMaker Studio by continuing with the next task.

Refer to [Amazon AI Fairness and Explainability Whitepaper](./Amazon.AI.Fairness.and.Explainability.Whitepaper.pdf) for more information about the SageMaker Clarify explainability process.

In [ ]:
#run-explainability-report

clarify_processor.run_explainability(
    data_config=explainability_data_config,
    model_config=model_config,
    explainability_config=shap_config,
)

### Task 2.3.3: Access the explainability report

As with the bias report, you can review the explainability report in Studio as part of the experiments.

1. Select the **Experiments** tab.

2. In the **Experiments** tab, select **Unassigned runs** from the left.

3. In the **Unassigned runs** list, select the training job name that contains **clarify-bias-** in the title. 

4. In the **Experiments** tab, select **Explainability** from the left.

5. When the explainability job is finished, to view the **Feature Importance** chart, choose <span style="background-color:#57c4f8; font-size:90%;  color:black; position:relative; top:-1px; padding-top:3px; padding-bottom:3px; padding-left:10px; padding-right:10px; border-color:#00a0d2; border-radius:2px; margin-right:5px; white-space:nowrap">View sample notebook</span>.

SageMaker Studio opens **Fairness and Explainability with SageMaker Clarify** notebook in a tab.

What features have the highest and lowest importance? Are there any results that are surprising?

### Conclusion

Congratulations! You have used SageMaker Clarify to create bias and explainability reports that you can use to develop more trustworthy and compliant models. In the next lab, you deploy your models and run inference. You continue working with your model in the next lab.

### Cleanup

You have completed this notebook. To move to the next part of the lab, do the following:

- Close this notebook file.
- Return to the lab session and continue with the **Conclusion**.